In [1]:
import numpy as np
import pandas as pd

In [2]:
X_train = pd.read_csv("X_train.csv").set_index('enrollee_id')
Y_train = pd.read_csv("Y_train.csv").set_index('enrollee_id').values.flatten()
X_test = pd.read_csv("X_test.csv").set_index('enrollee_id')
Y_test = pd.read_csv("Y_test.csv").set_index('enrollee_id').values.flatten()

In [3]:
selected_col = ['city_development_index', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'experience', 'company_size',
       'last_new_job', 'training_hours', 'company_type_Pvt Ltd']

X_train = X_train[selected_col] 
X_test = X_test[selected_col]

In [4]:
from sklearn import svm
from sklearn import metrics

In [5]:
def training(X_train, X_test, y_train, y_test, model): 
    # training process
    model.fit(X_train, y_train)
    
    # testing
    y_pred = model.predict_proba(X_test)[:,1]
    y_pred_train = model.predict_proba(X_train)[:,1]
    #print(y_test.to_numpy())
    #print(y_pred)
    
    test_score = metrics.roc_auc_score(y_test, y_pred)
    train_score = metrics.roc_auc_score(y_train, y_pred_train)
    return {'test_score':test_score, 'train_score':train_score}

In [6]:
model = {}
for C in (0.5,1,10,100,1000,5000,10000):
    print("C= ",C)
    model[C] = svm.SVC(C=C,kernel='rbf', gamma='scale',probability = True)
    print(training(X_train, X_test, Y_train, Y_test, model[C]))

C=  0.5
{'test_score': 0.5860847740218107, 'train_score': 0.6033722360438448}
C=  1
{'test_score': 0.6375853321693333, 'train_score': 0.6493348362705966}
C=  10
{'test_score': 0.707041782351089, 'train_score': 0.7380006306250926}
C=  100
{'test_score': 0.7229987191229929, 'train_score': 0.7549322496066474}
C=  1000
{'test_score': 0.7240355854824397, 'train_score': 0.7556607259845796}
C=  5000
{'test_score': 0.7244672553453358, 'train_score': 0.7556207379665935}
C=  10000
{'test_score': 0.7252114520422475, 'train_score': 0.7564570026695}


In [14]:
test_df = pd.read_csv('test_processed.csv').set_index('enrollee_id')[selected_col]
predict = model[1000].predict_proba(test_df)[:,1]
target_values = np.load("jobchange_test_target_values.npy")
score = metrics.roc_auc_score(target_values,predict)
print("AUC Score: ",score)

AUC Score:  0.747008478214209
